In [1]:
# %%
# ==============================================================
# 🧠 Self-Tuning FAISS Auto-Tuner (VDTuner-style)
# ==============================================================
import os
import gc
import time
import random
import numpy as np
import h5py
import faiss
import optuna
from optuna.samplers import NSGAIISampler
import plotly.graph_objects as go

# --------------------------------------------------------------
# Reproducibility (optional)
# --------------------------------------------------------------
np.random.seed(42)
random.seed(42)

# ==============================================================
# STEP 1 — Load Dataset
# ==============================================================
glove_path = "glove-100-angular.hdf5"  # http://ann-benchmarks.com/glove-100-angular.hdf5

print("📦 Loading GloVe 100D benchmark dataset …")
glove = h5py.File(glove_path, "r")

database_vectors = np.array(glove["train"])            # (1,183,514, 100)
query_vectors     = np.array(glove["test"])            # (10,000, 100)
ground_truth_idx  = np.array(glove["neighbors"])[:, :10]  # Recall@10 ground truth

# ==============================================================
# ⚙️ MODE SETTINGS
# ==============================================================
FAST_MODE = True  # 🚀 True = use 1K queries; False = use full 10K
if FAST_MODE:
    query_vectors = query_vectors[:1000]
    ground_truth_idx = ground_truth_idx[:1000]
    print("⚡ FAST_MODE: using 1,000 queries for tuning.")
else:
    print("🧠 FULL MODE: using all 10,000 queries for benchmark.")

print(f"✅ Loaded → DB: {database_vectors.shape}, Queries: {query_vectors.shape}")

# ==============================================================
# STEP 2 — Safe Normalization for Cosine Similarity
# ==============================================================
# Normalize to unit length (cosine ≡ inner product on normalized vectors)
database_vectors = database_vectors / np.clip(
    np.linalg.norm(database_vectors, axis=1, keepdims=True), 1e-10, None
)
query_vectors = query_vectors / np.clip(
    np.linalg.norm(query_vectors, axis=1, keepdims=True), 1e-10, None
)

n_neighbors = 10
num_database_vectors = len(database_vectors)
num_query_vectors = len(query_vectors)

# Use all CPU cores
faiss.omp_set_num_threads(os.cpu_count())

/opt/anaconda3/envs/vectordb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📦 Loading GloVe 100D benchmark dataset …
⚡ FAST_MODE: using 1,000 queries for tuning.
✅ Loaded → DB: (1183514, 100), Queries: (1000, 100)


In [3]:
# ==============================================================
# 🧠 Self-Tuning FAISS Auto-Tuner (VDTuner-style) — UPDATED for HNSW
# ==============================================================

# ... [previous imports & setup] ...

def objective(trial):
    """VDTuner-inspired FAISS objective with adaptive parameter search (fixed dynamic space)."""
    d = database_vectors.shape[1]

    # Choose index type
    index_type = trial.suggest_categorical("index_type", ["IVFFlat", "IVFPQ", "HNSWFlat"])

    nlist_max = max(512, min(2048, int(np.sqrt(num_database_vectors)) * 2))
    if index_type in ["IVFFlat", "IVFPQ"]:
        nlist  = trial.suggest_int("nlist", 256, nlist_max)
        nprobe = trial.suggest_int("nprobe", 8, min(512, nlist))
    else:
        nlist, nprobe = None, None

    quantizer = faiss.IndexFlatIP(d)

    # -------------------------------------------------
    # Build Index
    # -------------------------------------------------
    if index_type == "IVFFlat":
        index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_INNER_PRODUCT)

    elif index_type == "IVFPQ":
        pq_mode = trial.suggest_categorical("pq_mode", ["accurate", "fast"])
        pq_m    = trial.suggest_categorical("pq_m", [4, 5, 10, 20, 25, 50])
        pq_bits = trial.suggest_categorical("pq_bits", [4, 6, 8])

        if d % pq_m != 0:
            raise optuna.TrialPruned()

        if pq_mode == "accurate":
            pq_bits = 8
            if pq_m not in [25, 50]:
                pq_m = 50

        index = faiss.IndexIVFPQ(quantizer, d, nlist, pq_m, pq_bits, faiss.METRIC_INNER_PRODUCT)

    elif index_type == "HNSWFlat":
        hnsw_m  = trial.suggest_int("hnsw_m", 16, 64)
        ef_c    = trial.suggest_int("hnsw_efConstruction", 100, 400)
        ef_s    = trial.suggest_int("hnsw_efSearch", 100, 400)
        index = faiss.IndexHNSWFlat(d, hnsw_m)
        index.hnsw.efConstruction = ef_c
        index.hnsw.efSearch = ef_s
        # DO NOT call make_direct_map for HNSWFlat

    # -------------------------------------------------
    # Train & Add
    # -------------------------------------------------
    try:
        if not index.is_trained:
            index.train(database_vectors)
        # Only call make_direct_map for IVF indices
        if index_type in ["IVFFlat", "IVFPQ"]:
            index.make_direct_map()
        index.add(database_vectors)
    except Exception as e:
        print(f"[⚠️] Skipping invalid trial ({index_type}): {e}")
        del index
        gc.collect()
        return 0, 0

    if hasattr(index, "nprobe") and nprobe is not None:
        index.nprobe = nprobe

    # -------------------------------------------------
    # Warm-up + QPS Measurement
    # -------------------------------------------------
    _ = index.search(query_vectors[:10], k=n_neighbors)  # warm-up
    start_time = time.time()
    distances, indices = index.search(query_vectors, k=n_neighbors)
    total_time = time.time() - start_time
    qps = num_query_vectors / total_time if total_time > 0 else float("inf")

    # -------------------------------------------------
    # Recall Computation
    # -------------------------------------------------
    total_recall = 0
    for i in range(num_query_vectors):
        true_n = set(ground_truth_idx[i])
        retrieved_n = set(indices[i])
        total_recall += len(true_n.intersection(retrieved_n)) / n_neighbors
    avg_recall = total_recall / num_query_vectors

    # Cleanup
    del index, distances, indices
    gc.collect()

    return qps, avg_recall


In [4]:
# ==============================================================
# STEP 4 — Run NSGA-II Optimization (Evolutionary Search)
# ==============================================================
print("\n🚀 Starting FAISS adaptive tuning (NSGA-II)…")
optuna.logging.set_verbosity(optuna.logging.WARNING)

study = optuna.create_study(
    study_name="glove_faiss_adaptive_tuning",
    directions=["maximize", "maximize"],  # maximize QPS & Recall
    sampler=NSGAIISampler(population_size=24),
)

# Optional: warm-start with a few good prior configs (enqueue)
study.enqueue_trial({"index_type": "IVFFlat", "nlist": 540, "nprobe": 120})
study.enqueue_trial({"index_type": "IVFPQ",   "nlist": 820, "nprobe": 110, "pq_mode":"accurate", "pq_m":50})
study.enqueue_trial({"index_type": "HNSWFlat","hnsw_m": 32, "hnsw_efConstruction": 200, "hnsw_efSearch": 250})

n_trials = 40  # increase to 80–120 for paper-quality fronts
print(f"🔍 Using {num_query_vectors} queries; trials = {n_trials}")
study.optimize(objective, n_trials=n_trials)

print("\n✅ Tuning completed!\n")
for trial in study.best_trials:
    print(f"Params: {trial.params} | QPS: {trial.values[0]:.2f} | Recall: {trial.values[1]:.4f}")

# ==============================================================
# STEP 5 — Visualize Pareto Front (Color-coded by Index Type)
# ==============================================================
all_trials = [t for t in study.trials if t.values and "index_type" in t.params]
index_types = sorted(set(t.params["index_type"] for t in all_trials))
colors = {"IVFFlat": "blue", "IVFPQ": "green", "HNSWFlat": "orange"}

fig = go.Figure()
for idx_type in index_types:
    xs = [t.values[1] for t in all_trials if t.params["index_type"] == idx_type]  # recall
    ys = [t.values[0] for t in all_trials if t.params["index_type"] == idx_type]  # qps
    fig.add_trace(go.Scatter(
        x=xs, y=ys, mode="markers",
        name=idx_type,
        marker=dict(color=colors.get(idx_type, "gray"), size=6)
    ))

pareto_trials = study.best_trials
fig.add_trace(go.Scatter(
    x=[t.values[1] for t in pareto_trials],
    y=[t.values[0] for t in pareto_trials],
    mode="markers+lines",
    name="Pareto Front",
    marker=dict(color="red", size=10, symbol="star")
))

fig.update_layout(
    title="VDTuner-style FAISS Auto-Tuning — Recall vs QPS Pareto Front",
    xaxis_title="Recall",
    yaxis_title="QPS (Queries per Second)",
    hovermode="closest",
)

try:
    fig.show()
except Exception:
    fig.write_html("faiss_pareto.html")
    print("📊 Plot saved as faiss_pareto.html")

# ==============================================================
# STEP 6 — Export Results (Optional)
# ==============================================================
try:
    df = study.trials_dataframe()
    df.to_csv("faiss_tuning_results.csv", index=False)
    print("📝 Saved all trial results → faiss_tuning_results.csv")
except Exception as e:
    print(f"Skip saving CSV (install pandas if needed): {e}")
# %%


🚀 Starting FAISS adaptive tuning (NSGA-II)…
🔍 Using 1000 queries; trials = 40

✅ Tuning completed!

Params: {'index_type': 'HNSWFlat', 'hnsw_m': 32, 'hnsw_efConstruction': 200, 'hnsw_efSearch': 250} | QPS: 5341.16 | Recall: 0.9387
Params: {'index_type': 'IVFFlat', 'nlist': 772, 'nprobe': 144} | QPS: 707.03 | Recall: 0.9811
Params: {'index_type': 'IVFFlat', 'nlist': 1034, 'nprobe': 405} | QPS: 339.53 | Recall: 0.9958
Params: {'index_type': 'IVFPQ', 'nlist': 1167, 'nprobe': 10, 'pq_mode': 'fast', 'pq_m': 25, 'pq_bits': 8} | QPS: 29380.72 | Recall: 0.5041
Params: {'index_type': 'IVFFlat', 'nlist': 1945, 'nprobe': 408} | QPS: 599.05 | Recall: 0.9877
Params: {'index_type': 'IVFFlat', 'nlist': 1162, 'nprobe': 419} | QPS: 354.38 | Recall: 0.9954
Params: {'index_type': 'IVFFlat', 'nlist': 909, 'nprobe': 444} | QPS: 248.79 | Recall: 0.9975
Params: {'index_type': 'IVFFlat', 'nlist': 616, 'nprobe': 407} | QPS: 174.07 | Recall: 0.9992
Params: {'index_type': 'HNSWFlat', 'hnsw_m': 38, 'hnsw_efConst

Skip saving CSV (install pandas if needed): Tried to import 'pandas' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'pandas'.
